In [ ]:
!pip install transformers torch accelerate

In [1]:
# loading model from huggingface repo

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch, json, requests

MODEL_NAME = "PavanB99Indian/mbti-classification-distilbert"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MAX_LENGTH = 128

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME).to(DEVICE)
model.eval()

label_url = f"https://huggingface.co/{MODEL_NAME}/resolve/main/label_mapping.json"
label_mapping = json.loads(requests.get(label_url).text)["classes"]

label2id = {c:i for i,c in enumerate(label_mapping)}
id2label = {i:c for c,i in label2id.items()}

print("Model Loaded Successfully ")
print("Labels:", id2label)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Model Loaded Successfully 
Labels: {0: 'ENFJ', 1: 'ENFP', 2: 'ENTJ', 3: 'ENTP', 4: 'ESFJ', 5: 'ESFP', 6: 'ESTJ', 7: 'ESTP', 8: 'INFJ', 9: 'INFP', 10: 'INTJ', 11: 'INTP', 12: 'ISFJ', 13: 'ISFP', 14: 'ISTJ', 15: 'ISTP'}


In [2]:
# function to predict personality type from input text

def predict(text):
    if len(text.strip()) == 0:
        return " Empty input"

    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=MAX_LENGTH
    ).to(DEVICE)

    with torch.no_grad():
        logits = model(**inputs).logits

    probs = torch.softmax(logits, dim=1).cpu().numpy()[0]

    pred_idx = int(probs.argmax())
    pred_label = id2label[pred_idx]
    confidence = float(probs[pred_idx])

    return {
        "Predicted Type": pred_label,
        "Confidence": round(confidence, 4),
    }


In [4]:
sample = "I love deep discussions and spending time alone thinking."
result = predict(sample)
result


{'Predicted Type': 'INFP', 'Confidence': 0.3916}

In [7]:
texts = [
    "I enjoy exploring new ideas",
    "I love hanging out with lots of people",
    "I plan everything and hate uncertainty"
]

for t in texts:
    print(t, " : ", predict(t)["Predicted Type"])


I enjoy exploring new ideas  :  INFP
I love hanging out with lots of people  :  INFP
I plan everything and hate uncertainty  :  INTJ
